In [ ]:
import pandas as pd
import asyncio
import websockets
import json
import os
from datetime import datetime, timedelta
import nest_asyncio

nest_asyncio.apply()

FINNHUB_API_KEY = "d056jb9r01qoigrsmf5gd056jb9r01qoigrsmf60"

# S&P 500 tickers (limit to first 5 for demo)
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/master/data/constituents.csv"
tickers = pd.read_csv(url)['Symbol'].tolist()

# WebSocket endpoint
FINNHUB_WS_URL = f"wss://ws.finnhub.io?token={FINNHUB_API_KEY}"

# In-memory storage
df_all_data = pd.DataFrame()

# Create directory
os.makedirs("dati_stream", exist_ok=True)

async def save_data_periodically(interval_sec=60):
    global df_all_data
    while True:
        await asyncio.sleep(interval_sec)
        if not df_all_data.empty:
            timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")
            path = f"dati_stream/finnhub_stream_{timestamp}.csv"
            df_all_data.to_csv(path, index=False)
            print(f"💾 Salvato {len(df_all_data)} righe in {path}")

async def stream_finnhub(duration_seconds=60):
    global df_all_data

    async with websockets.connect(FINNHUB_WS_URL) as ws:
        # Subscribe to tickers
        for symbol in tickers:
            await ws.send(json.dumps({"type": "subscribe", "symbol": symbol}))
            print(f"📡 Subscribed to {symbol}")

        # Start saving data
        save_task = asyncio.create_task(save_data_periodically(30))

        end_time = datetime.utcnow() + timedelta(seconds=duration_seconds)

        try:
            while datetime.utcnow() < end_time:
                message = await ws.recv()
                data = json.loads(message)

                if data.get("type") == "trade":
                    for trade in data.get("data", []):
                        row = {
                            "Ticker": trade["s"],
                            "Timestamp": pd.to_datetime(trade["t"], unit="ms", utc=True),
                            "Price": trade["p"],
                            "Size": trade["v"]
                        }
                        df_all_data = pd.concat([df_all_data, pd.DataFrame([row])], ignore_index=True)
                        print(df_all_data.tail(1))
        finally:
            await ws.close()
            save_task.cancel()
            print("✅ Streaming finished.")

# Run the stream
await stream_finnhub(duration_seconds=60)


📡 Subscribed to MMM
📡 Subscribed to AOS
📡 Subscribed to ABT
📡 Subscribed to ABBV
📡 Subscribed to ACN
📡 Subscribed to ADBE
📡 Subscribed to AMD
📡 Subscribed to AES
📡 Subscribed to AFL
📡 Subscribed to A
📡 Subscribed to APD
📡 Subscribed to ABNB
📡 Subscribed to AKAM
📡 Subscribed to ALB
📡 Subscribed to ARE
📡 Subscribed to ALGN
📡 Subscribed to ALLE
📡 Subscribed to LNT
📡 Subscribed to ALL
📡 Subscribed to GOOGL
📡 Subscribed to GOOG
📡 Subscribed to MO
📡 Subscribed to AMZN
📡 Subscribed to AMCR
📡 Subscribed to AEE
📡 Subscribed to AEP
📡 Subscribed to AXP
📡 Subscribed to AIG
📡 Subscribed to AMT
📡 Subscribed to AWK
📡 Subscribed to AMP
📡 Subscribed to AME
📡 Subscribed to AMGN
📡 Subscribed to APH
📡 Subscribed to ADI
📡 Subscribed to ANSS
📡 Subscribed to AON
📡 Subscribed to APA
📡 Subscribed to APO
📡 Subscribed to AAPL
📡 Subscribed to AMAT
📡 Subscribed to APTV
📡 Subscribed to ACGL
📡 Subscribed to ADM
📡 Subscribed to ANET
📡 Subscribed to AJG
📡 Subscribed to AIZ
📡 Subscribed to T
📡 Subscribed to ATO
📡 Subsc

C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:48: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  end_time = datetime.utcnow() + timedelta(seconds=duration_seconds)
C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


  Ticker                        Timestamp   Price  Size
0   AAPL 2025-05-09 13:30:14.663000+00:00  198.95   100
  Ticker                        Timestamp  Price  Size
1      T 2025-05-09 13:30:48.294000+00:00  27.43   200
  Ticker                        Timestamp  Price  Size
2      T 2025-05-09 13:30:48.294000+00:00  27.43   200


C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


  Ticker                        Timestamp   Price  Size
3      T 2025-05-09 13:30:57.065000+00:00  27.445   400
  Ticker                        Timestamp  Price  Size
4   ADBE 2025-05-09 13:31:05.597000+00:00  384.0   100
  Ticker                        Timestamp  Price  Size
5    ADI 2025-05-09 13:30:48.306000+00:00  209.3   100
  Ticker                        Timestamp   Price  Size
6   ADBE 2025-05-09 13:31:05.599000+00:00  383.98   100
  Ticker                        Timestamp   Price  Size
7  GOOGL 2025-05-09 13:30:14.774000+00:00  154.51   100
  Ticker                        Timestamp   Price  Size
8  GOOGL 2025-05-09 13:30:14.774000+00:00  154.51   100
  Ticker                        Timestamp   Price  Size
9      T 2025-05-09 13:31:05.308000+00:00  27.435   100
   Ticker                        Timestamp   Price  Size
10   AAPL 2025-05-09 13:30:48.601000+00:00  199.45   100
   Ticker                        Timestamp   Price  Size
11   AAPL 2025-05-09 13:30:48.601000+00:00  199.4

...

C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:
C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:31: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime("%Y%m%d_%H%M%S")


💾 Salvato 243 righe in dati_stream/finnhub_stream_20250509_133358.csv
    Ticker                        Timestamp   Price  Size
243  GOOGL 2025-05-09 13:31:37.790000+00:00  154.35   100
    Ticker                        Timestamp   Price  Size
244  GOOGL 2025-05-09 13:31:37.790000+00:00  154.35   100
    Ticker                        Timestamp   Price  Size
245   AAPL 2025-05-09 13:33:17.074000+00:00  200.28   100
    Ticker                        Timestamp   Price  Size
246   AAPL 2025-05-09 13:33:17.075000+00:00  200.28   100
    Ticker                        Timestamp   Price  Size
247   AAPL 2025-05-09 13:31:37.818000+00:00  199.81   100
    Ticker                        Timestamp   Price  Size
248    AMD 2025-05-09 13:31:38.106000+00:00  103.06   100
    Ticker                        Timestamp   Price  Size
249    AMD 2025-05-09 13:33:17.213000+00:00  103.32   150
    Ticker                        Timestamp   Price  Size
250      T 2025-05-09 13:31:11.394000+00:00  27.455   600


C:\Users\miklo\AppData\Local\Temp\ipykernel_7492\3439354500.py:51: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  while datetime.utcnow() < end_time:


✅ Streaming finished.


: 